In [1]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import scipy.optimize as solver
import datetime as dt
from functools import reduce

import cvxopt as opt
from cvxopt import blas, solvers
from scipy.stats import norm

In [2]:
pip install yfinance

In [3]:
import yfinance as yf
#投資標的
Closeprice = pd.DataFrame()
tickers = ["2801.TW","2880.TW","2881.TW","2882.TW","2883.TW","2884.TW","2885.TW","2886.TW","2887.TW","2890.TW","2891.TW","2892.TW"]


for i in tickers:
  tmp = yf.download(i, start="2010-01-01", end=dt.date.today())
  Closeprice[i] = tmp['Adj Close']
returns = np.log(Closeprice / Closeprice.shift(1))

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [4]:
tmp = yf.download("2801.TW", start="2010-01-01", end=dt.date.today())
tmp['Adj Close']

[*********************100%%**********************]  1 of 1 completed


Date
2010-01-04     6.444738
2010-01-05     6.444738
2010-01-06     6.529538
2010-01-07     6.508336
2010-01-08     6.571937
                ...    
2023-11-16    17.650000
2023-11-17    17.700001
2023-11-20    17.799999
2023-11-21    17.900000
2023-11-22    17.750000
Name: Adj Close, Length: 3402, dtype: float64

In [5]:
returns

,2801.TW,2880.TW,2881.TW,2882.TW,2883.TW,2884.TW,2885.TW,2886.TW,2887.TW,2890.TW,2891.TW,2892.TW
Date,,,,,,,,,,,,
2010-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-05,0.000000,0.007491,0.006357,-0.001682,0.008048,-0.003752,-0.012931,0.010695,0.027399,-0.003945,0.000000,0.007566
2010-01-06,0.013072,0.017263,0.031194,0.016695,0.002002,0.014926,0.021460,0.010582,0.011517,0.011788,0.029559,0.012485
2010-01-07,-0.003252,0.000000,-0.016099,-0.009983,-0.002002,0.011050,-0.017131,-0.010582,-0.019269,-0.019725,-0.009756,-0.015000
2010-01-08,0.009724,-0.002448,0.004981,0.009983,0.011952,0.014546,0.010741,0.007947,0.007752,0.003976,0.004890,0.005025
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-16,0.005682,0.002328,0.003185,-0.002188,0.012632,0.022111,0.005911,0.007682,0.002774,0.010638,0.001864,0.012809
2023-11-17,0.002829,0.004640,0.004758,0.003280,0.000000,0.000000,-0.003937,0.000000,-0.002774,-0.002649,0.000000,0.001817
2023-11-20,0.005634,0.000000,0.004736,-0.002186,-0.004193,0.001986,0.000000,0.000000,0.002774,-0.005319,0.000000,-0.003636


tickers = ["1316.TW",
"1436.TW",
"1438.TW",
"1442.TW",
"1805.TW",
"1808.TW",
"2501.TW",
"2504.TW",
"2505.TW",
"2506.TW",
"2509.TW",
"2511.TW",
"2515.TW",
"2516.TW",
"2520.TW",
"2524.TW",
"2527.TW",
"2528.TW",
"2530.TW",
"2534.TW",
"2535.TW",
"2536.TW",
"2537.TW",
"2538.TW",
"2539.TW",
"2540.TW",
"2542.TW",
"2543.TW",
"2545.TW",
"2546.TW",
"2547.TW",
"2548.TW",
"2596.TW",
"2597.TW",
"2841.TW",
"2923.TW",
"3052.TW",
"3056.TW",
"3266.TW",
"3489.TW",
"3703.TW",
"4113.TW",
"4416.TW",
"4907.TW",
"5206.TW",
"5213.TW",
"5324.TW",
"5455.TW",
"5508.TW",
"5511.TW",
"5512.TW",
"5514.TW",
"5515.TW",
"5516.TW",
"5519.TW",
"5520.TW",
"5521.TW",
"5522.TW",
"5523.TW",
"5525.TW",
"5529.TW",
"5531.TW",
"5533.TW",
"5534.TW",
"5543.TW",
"5546.TW",
"6171.TW",
"6177.TW",
"6186.TW",
"6212.TW",
"6219.TW",
"6264.TW",
"8424.TW",
"9906.TW",
"9946.TW"]

In [6]:
#收盤價為Series格式
Closeprice.values[-1]

array([17.75      , 21.79999924, 64.59999847, 46.29999924, 12.30000019,
       25.5       , 25.60000038, 39.84999847, 18.10000038, 18.85000038,
       27.20000076, 27.70000076])

In [ ]:
#輸入投入金額
invests=[]
for i in range(len(tickers)):
  invest=int(input(f'Investing in {tickers[i]} Dollar:'))
  invests.append(invest)
wts =[i/sum(invests) for i in invests]

Investing in 2801.TW Dollar:1
Investing in 2880.TW Dollar:1
Investing in 2881.TW Dollar:1
Investing in 2882.TW Dollar:1
Investing in 2883.TW Dollar:1
Investing in 2884.TW Dollar:1
Investing in 2885.TW Dollar:1
Investing in 2886.TW Dollar:1
Investing in 2887.TW Dollar:1
Investing in 2890.TW Dollar:1
Investing in 2891.TW Dollar:1
Investing in 2892.TW Dollar:1


In [7]:
# 輸入投資單位數
units=[]
for j in range(len(tickers)):
  unit=int(input("Investing Unit:"))
  units.append(unit)
wts =[i/sum(units) for i in units]

#收盤價為Series格式
position = units * np.transpose(Closeprice.values[-1])
position

Investing Unit:100
Investing Unit:100
Investing Unit:100
Investing Unit:100
Investing Unit:100
Investing Unit:100
Investing Unit:100
Investing Unit:100
Investing Unit:100
Investing Unit:100
Investing Unit:100
Investing Unit:100


array([1775.        , 2179.99992371, 6459.99984741, 4629.99992371,
       1230.00001907, 2550.        , 2560.00003815, 3984.99984741,
       1810.00003815, 1885.00003815, 2720.00007629, 2770.00007629])

In [8]:
# We calculate the covariance matrix. A one step process. Note that these are daily covariances.
# In order to get annual covariances, we will need to multiply by 250 (note that variances and covariances
# scale linearly with time, while standard deviation scales by the square root of time)
# Generate Var-Cov matrix
V= returns.cov() *250
V

,2801.TW,2880.TW,2881.TW,2882.TW,2883.TW,2884.TW,2885.TW,2886.TW,2887.TW,2890.TW,2891.TW,2892.TW
2801.TW,0.034957,0.022902,0.024746,0.025229,0.026761,0.023299,0.027162,0.023884,0.026139,0.025239,0.026030,0.023789
2880.TW,0.022902,0.028708,0.024481,0.024414,0.025778,0.023896,0.026030,0.022529,0.024057,0.024938,0.025030,0.023346
2881.TW,0.024746,0.024481,0.053813,0.040769,0.034510,0.027351,0.034221,0.028279,0.030525,0.030977,0.033814,0.024629
2882.TW,0.025229,0.024414,0.040769,0.052842,0.035599,0.026540,0.034934,0.027313,0.030198,0.030622,0.033376,0.024223
2883.TW,0.026761,0.025778,0.034510,0.035599,0.055175,0.028382,0.035697,0.027521,0.031916,0.032570,0.032825,0.026025
2884.TW,0.023299,0.023896,0.027351,0.026540,0.028382,0.045197,0.029606,0.025479,0.027615,0.027299,0.029099,0.024236
2885.TW,0.027162,0.026030,0.034221,0.034934,0.035697,0.029606,0.055048,0.028362,0.031080,0.031796,0.034583,0.026801
2886.TW,0.023884,0.022529,0.028279,0.027313,0.027521,0.025479,0.028362,0.039443,0.025848,0.026953,0.029601,0.023603
2887.TW,0.026139,0.024057,0.030525,0.030198,0.031916,0.027615,0.031080,0.025848,0.046206,0.029639,0.030746,0.024686
2890.TW,0.025239,0.024938,0.030977,0.030622,0.032570,0.027299,0.031796,0.026953,0.029639,0.045949,0.030321,0.024961


In [9]:
# We calculate the covariances of asset i to portfolio p.
cov_ip = V .dot(wts)
cov_ip

2801.TW    0.025845
2880.TW    0.024676
2881.TW    0.032343
2882.TW    0.032172
2883.TW    0.032730
2884.TW    0.028166
2885.TW    0.032943
2886.TW    0.027401
2887.TW    0.029888
2890.TW    0.030105
2891.TW    0.031662
2892.TW    0.025170
dtype: float64

In [10]:
# Calculate the portfolio volatility, in percent, and in dollars. Note that we
# are multiplying by sqrt(250) to annualize the daily volatilities.
vol_p_pct = ((np.transpose(wts).dot(V).dot(wts))) ** 0.5 # in percent, annual Var
vol_p = ((np.transpose(position).dot(V).dot(position)))** 0.5 #in dollars
vol_p

6052.640800812978

In [11]:
# Calculate the VaR, in dollars and percent. Note that 'qnorm' is equivalent to Excel's NORMSINV.
# Note that we are calculating things here at the 95% confidence level.
confident_level = 0.95

var_p = norm.ppf(confident_level) * vol_p #Portfolio VaR in dollars
var_p_pct = norm.ppf(confident_level) * vol_p_pct #Portfolio VaR in percent

In [12]:
# Calculate the volatility of the i-th asset. We can then calculate the VaR of the i-th asset.
vol_i = (np.diag(V)) ** 0.5
vol_i

array([0.18696683, 0.16943542, 0.23197735, 0.2298742 , 0.23489372,
       0.2125968 , 0.23462258, 0.19860238, 0.214956  , 0.21435603,
       0.22040151, 0.17261152])

In [13]:
VaRi = norm.ppf(confident_level) * vol_i * position
VaRi

array([ 545.87121049,  607.55827926, 2464.9342636 , 1750.64651081,
        475.22993301,  891.71107808,  987.95471863, 1301.78727767,
        639.96382007,  664.6214194 ,  986.07681287,  786.46032018])

In [14]:
# We calculate the undiversified VaR as the sum of the VaRs of the individual assets.
uVar = sum(VaRi)
uVar

12102.815644059086

In [15]:
# We calculate the beta. Note that this is the same answer
# you would get if you did beta = sum(w) * (V %*% w)/as.numeric(vol_p^2)
beta = sum(position) * (V.dot(position))/(vol_p.item(0) **2)
beta

2801.TW    0.827472
2880.TW    0.798709
2881.TW    1.150766
2882.TW    1.118710
2883.TW    1.050090
2884.TW    0.913268
2885.TW    1.080587
2886.TW    0.917529
2887.TW    0.961726
2890.TW    0.972862
2891.TW    1.048513
2892.TW    0.816553
dtype: float64

In [16]:
# Calculate Marginal VaR
mvar = norm.ppf(confident_level) *  (V.dot(position))/vol_p.item(0)
mvar

2801.TW    0.238405
2880.TW    0.230118
2881.TW    0.331549
2882.TW    0.322314
2883.TW    0.302544
2884.TW    0.263123
2885.TW    0.311330
2886.TW    0.264351
2887.TW    0.277085
2890.TW    0.280293
2891.TW    0.302089
2892.TW    0.235259
dtype: float64

In [17]:
# Calculate Component VaR
cvar = mvar * position
cvar


2801.TW     423.168203
2880.TW     501.656508
2881.TW    2141.809663
2882.TW    1492.312810
2883.TW     372.128522
2884.TW     670.964534
2885.TW     797.004875
2886.TW    1053.438569
2887.TW     501.523304
2890.TW     528.352473
2891.TW     821.682371
2892.TW     651.666343
dtype: float64

In [18]:
PCTtoContribution = cvar/ sum(cvar)
PCTtoContribution

2801.TW    0.042505
2880.TW    0.050389
2881.TW    0.215134
2882.TW    0.149895
2883.TW    0.037378
2884.TW    0.067395
2885.TW    0.080055
2886.TW    0.105813
2887.TW    0.050375
2890.TW    0.053070
2891.TW    0.082534
2892.TW    0.065457
dtype: float64

In [19]:
position=pd.DataFrame(position, index=tickers)
print(position)
units=pd.DataFrame(units, index=tickers)
print(units)

                   0
2801.TW  1775.000000
2880.TW  2179.999924
2881.TW  6459.999847
2882.TW  4629.999924
2883.TW  1230.000019
2884.TW  2550.000000
2885.TW  2560.000038
2886.TW  3984.999847
2887.TW  1810.000038
2890.TW  1885.000038
2891.TW  2720.000076
2892.TW  2770.000076
           0
2801.TW  100
2880.TW  100
2881.TW  100
2882.TW  100
2883.TW  100
2884.TW  100
2885.TW  100
2886.TW  100
2887.TW  100
2890.TW  100
2891.TW  100
2892.TW  100


In [20]:
Risk = pd.concat([ units ,position,beta, mvar ,cvar,PCTtoContribution],axis=1, names=['Investing Units','Market Value','Beta of each stock','Marginal VaR','Component VaR','PCTtoContribution'])
Risk.columns=['Investing Units','Market Value','Beta of each stock','Marginal VaR','Component VaR','PCTtoContribution']
Risk

,Investing Units,Market Value,Beta of each stock,Marginal VaR,Component VaR,PCTtoContribution
2801.TW,100,1775.000000,0.827472,0.238405,423.168203,0.042505
2880.TW,100,2179.999924,0.798709,0.230118,501.656508,0.050389
2881.TW,100,6459.999847,1.150766,0.331549,2141.809663,0.215134
2882.TW,100,4629.999924,1.118710,0.322314,1492.312810,0.149895
2883.TW,100,1230.000019,1.050090,0.302544,372.128522,0.037378
2884.TW,100,2550.000000,0.913268,0.263123,670.964534,0.067395
2885.TW,100,2560.000038,1.080587,0.311330,797.004875,0.080055
2886.TW,100,3984.999847,0.917529,0.264351,1053.438569,0.105813
2887.TW,100,1810.000038,0.961726,0.277085,501.523304,0.050375
2890.TW,100,1885.000038,0.972862,0.280293,528.352473,0.053070
